# Image Classification with pytorch

## Pytorch and Data Loaders

Pytorch has developed standard conventions of interacting with data.  
The two main conventions of interacting with data are `datasets` and `data loaders`  
**dataset** is a python class that allows us to get the data we'r supplying to the neural network.  
**dala loader** is what feeds data from dataset into the network (This can encomepass information about howmany worker processes are feeding data into the network? or howmay images we are passing at once?).  

### Dataset
Every dataset (images, audio, text, 3D landscape, stock marketinformation etc..) can interact with pytorch if it satisfies this abstract python class
```
class Dataset(object):
    def __getitem__(self, index):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError
```

* A method that retrieves the size of our dataset 
* A method that can retrieve an item from our dataset 

### Building and training a Dataset

`torchvision` includes a package called `ImageFolder` that does pretymuch everything, providing our images in a structure where each directory is a label

In [1]:
import torchvision 
from torchvision import transforms 

In [2]:
train_path = "/Users/sachitanandp/Documents/datasets/image/fish_cat/train"

In [3]:
transforms = transforms.Compose([
                                transforms.Resize(64), 
                                transforms.ToTensor(), 
                                transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                                    std=[0.229, 0.224, 0.225])
                                ])

In [4]:
train_data = torchvision.datasets.ImageFolder(root=train_path, transform=transforms)

torchvision also allows you to specify a list of transforms that will be applied to an image before it gets fed into the neural network.  
The default transform is to take image data and turn it into a tensor `transforms.ToTensor()`.  
Normalising is important bcoz a lot of multiplication will be happening as inputs pass through the layer of the neural network; keeping the incoming values between 0 and 1 prevents the values from getting too large during training phase (known as exploding gradient problem). And that magic incarnation is just the mean and standard deviation of `imagenet` dataset as a whole. You could calculate it for fish and cats separately but these values are decent enough. (If you were working on a completely different dataset, you’d have to calculate that mean and deviation, although many people just use these ImageNet constants and report acceptable results.) 

The composable transforms allow us to easily do things like image rotation and skewing for data augmentation

**Note**
*We’re resizing the images to 64 × 64 in this example. I’ve made that arbitrary choice in order to make the computation in our upcoming first network fast. Most existing architectures that you’ll see in Chapter 3 use 224 × 224 or 299 × 299 for their image inputs. In general, the larger the input size, the more data for the network to learn from. The flip side is that you can often fit a smaller batch of images within the GPU’s memory.*

### Building Validation and Test Datasets

The training data is setup and the same is to be repeated for our validation data.  

In [5]:
val_path = "/Users/sachitanandp/Documents/datasets/image/fish_cat/val"
val_data = torchvision.datasets.ImageFolder(root=val_path, 
                                           transform=transforms)

In [6]:
test_path = "/Users/sachitanandp/Documents/datasets/image/fish_cat/test"
test_data = torchvision.datasets.ImageFolder(root=test_path, 
                                            transform=transforms)

* Training set: Used in the training pass to update the model  
* Validation set: Used to evaluate how the model is generalizing to the problem domain, rather than fitting to the training data; not used to update the model directly  
* Test set: A final dataset that provides a final evaluation of the model’s performance after training is complete

In [7]:
from torch.utils import data

In [8]:
batch_size=64
train_data_loader = data.DataLoader(train_data, batch_size=batch_size)
val_data_loader = data.DataLoader(val_data, batch_size=batch_size)
test_data_loader = data.DataLoader(test_data, batch_size=batch_size)

you can experiment with some of the additional parameters: you can specify how datasets are sampled, whether the entire set is shuffled on each run, and how many worker processes are used to pull data out of the dataset. This can all be found in the [PyTorch documentation](https://pytorch.org/docs/stable/data.html).

## Neural Network

### Activation Functions
Relu  
Softmax  

### Creating a network

Creating a network in pytorch is very pythonic. We inherit from a class called `torch.nn.Network` and fillout the `__init__` and `forward` methods

In [9]:
import torch.nn as nn
import torch.nn.functional as F

In [10]:
class SimpleNet(nn.Module):
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(12288, 64)
        self.fc2 = nn.Linear(84, 50)
        self.fc3 = nn.Linear(50, 2)
    
    def forward(self):
        x = x.view(-1, 12288)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.softmax(self.fc3(x))
        return x

In [11]:
simplenet = SimpleNet()

### Loss Functions
* CrossEntropyLoss: This loss function also incorporates the `softmax()` function as part of its operation. Hence our implimentation of forward function would change from `F.softmax(self.fc3(x))` to `self.fc3(x)`
* MSELoss

### Optimizing
To perform updates on a nn we use optimizer.  
PyTorch ships with SGD and others such as AdaGrad and RMSProp, as well as Adam, the optimizer we will be using for the majority of the book.  

One of the key improvements that Adam makes (as does RMSProp and AdaGrad) is that it uses a learning rate per parameter, and adapts that learning rate depending on the rate of change of those parameters. It keeps an exponentially decaying list of gradients and the square of those gradients and uses those to scale the global learning rate that Adam is working with. Adam has been empirically shown to outperform most other optimizers in deep learning networks, but you can swap out Adam for SGD or RMSProp or another optimizer to see if using a different technique yields faster and better training for your particular application.  


In [12]:
import torch.optim as optim 
optimizer = optim.Adam(simplenet.parameters(), lr=0.001)

## Training

In [13]:
for epoch in range(epochs):
    for batch in train_data_loader:
        optimizer.zero_grad()
        input, target = batch 
        output = model(input)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        

NameError: name 'epochs' is not defined

It turns out that calculated gradients accumlate by default, meaning that if we didnt zero the gradients at the end of the batch's iteration, the next batch would have to deal with this batch's gradients aswell as its own and the batch after that would have to cope up with the previous two.  


### Making it work on GPU

In [14]:
import torch

In [15]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
    
simplenet.to(device)

SimpleNet(
  (fc1): Linear(in_features=12288, out_features=64, bias=True)
  (fc2): Linear(in_features=84, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=2, bias=True)
)

**Note:**In earlier versions of PyTorch, you would use the cuda() method to copy data to the GPU instead. If you come across that method when looking at other people’s code, just be aware that it’s doing the same thing as to()!

### putting it all together

In [16]:
def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(epochs):
        training_loss = 0.0
        valid_loss = 0.0 
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, targets = batch 
            inputs = inputs.to(device)
            targets = targets.to(device)
            output = model(inputs)
            loss = loss_fn(output, target)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item()
        training_loss /= len(train_iterator)
        
        model.eval()
        num_correct = 0 
        num_examples = 0 
        for batch in val_loader:
            inputs, targets = batch 
            inputs = inputs.to(device)
            targets = targets.to(device)
            loss = loss_fn(output, target)
            valid_loss += loss.data.item()
            correct = (torch.eq(torch.max(F.softmax(output), dim=1)[1], target).view(-1))
            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(valid_iterator)
        print("Epoch: {}, Training Loss {:.2F}, Validation Loss {:.2F}, accuracy={:.2F}".format(epoch, training_loss, valid_loss, num_correct/num_examples))

In [17]:
train(simplenet, optimizer, torch.nn.CrossEntropyLoss(), train_data_loader, val_data_loader, device=device)

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 85 and 116 in dimension 3 at /Users/distiller/project/conda/conda-bld/pytorch_1573049287641/work/aten/src/TH/generic/THTensor.cpp:689